In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
# import logging
# logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]" #これは自分で作らなきゃいけないのかな
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [2]:
#自分でやってみる

# Tokenized input
text = r"I am a student." 


#単語とか活用とかそれらをいい感じに区切ってくれるやつ
tokenized_text = tokenizer.tokenize(text)
# [CLF]を先頭につけなければいけない、最後には[SEP]をつけなければ行けないが
tokenized_text= ['[CLS]']+tokenized_text+['[SEP]']
print(tokenized_text)

# 単語番号に変換するためのコンバーター
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

# Convert inputs to PyTorch tensors
# テンソル化
tokens_tensor = torch.tensor([indexed_tokens])

#文章が一つの場合にもセグメントテンサーが必要なのかも知れない
segments_tensors=torch.tensor([0]*len(indexed_tokens))

['[CLS]', 'i', 'am', 'a', 'student', '.', '[SEP]']
[101, 1045, 2572, 1037, 3076, 1012, 102]


Let's see how to use `BertModel` to get hidden states

In [3]:
# Load pre-trained model (weights)
# BERTのモデルを持ってくる
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cpu')
segments_tensors = segments_tensors.to('cpu')
model.to('cpu')

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased

In [4]:
len(encoded_layers)

12

In [5]:
encoded_layers[-1].shape

torch.Size([1, 7, 768])

encoded_layersの中身はもしかして、(bertの層(リスト), トークン数, hidden state) になっている？

In [5]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-large-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cpu')
segments_tensors = segments_tensors.to('cpu')
model.to('cpu')

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased

In [6]:
encoded_layers

[tensor([[[ 0.0209, -0.0933, -0.0899,  ..., -0.0462, -0.0979, -0.0050],
          [-0.1881,  0.6580,  0.2152,  ...,  0.5455, -0.0858,  0.4080],
          [-0.8712, -0.1690,  0.2003,  ...,  0.5357, -0.1346,  0.4288],
          ...,
          [-0.3829,  0.1071,  0.3474,  ...,  0.6633, -0.5993, -0.4130],
          [ 0.4142, -0.1914, -1.1008,  ...,  0.1034,  0.5236, -0.3538],
          [-0.2399,  0.0702, -0.6301,  ..., -0.2046,  0.1391,  0.2259]]]),
 tensor([[[ 0.1202, -0.0804, -0.0579,  ...,  0.1063, -0.1013,  0.0238],
          [-0.2258,  0.4770,  0.0755,  ...,  0.1937, -0.1232,  0.5070],
          [-0.8024, -0.1824,  0.1630,  ...,  0.4806,  0.1115,  0.3375],
          ...,
          [-0.6579,  0.0999,  0.4524,  ...,  0.4763, -0.7529, -0.1294],
          [ 0.3774, -0.1221, -1.1118,  ..., -0.1665,  0.5325, -0.3628],
          [-0.2749, -0.2171, -0.5670,  ..., -0.1113,  0.0711, -0.0443]]]),
 tensor([[[ 0.2405, -0.1409, -0.1571,  ...,  0.0893, -0.1792,  0.0927],
          [ 0.1499,  0.5117,

In [7]:
# 12レイヤー*

encoded_layers[-1].shape

torch.Size([1, 14, 1024])

In [8]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cpu')
segments_tensors = segments_tensors.to('cpu')
model.to('cpu')

# Predict hidden states features for each layer
with torch.no_grad():
    last_layers, _ = model(tokens_tensor, segments_tensors,output_all_encoded_layers=False)
    #最終層だけ取り出す。
# We have a hidden states for each of the 12 layers in model bert-base-uncased
# assert len(encoded_layers) == 1

In [9]:
torch.allclose(encoded_layers[-1],last_layers)

RuntimeError: The size of tensor a (1024) must match the size of tensor b (768) at non-singleton dimension 2

In [ ]:
encoded_layers[11].shape

In [ ]:
encoded_layers

いろいろな文の内部表現を取り出したい

In [ ]:
text = "[CLS] I like unko! [SEP] can I input several sentencies? [SEP] ok? [SEP]"
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])

In [ ]:
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor)


In [ ]:
encoded_layers[-1].shape